______

# Land Cover Classification for use in the CAPRA Model 

## Image Classification
_____

### Learning Objectives

In this lesson you will learn concepts of supervised classification. You will explore processes of training data collection, classifier selection, classifier training, image classification and accuracy assessment. 

The purpose of this tutorial is to derive a land cover map from satellite imagery using a classification method called recursive partitioning. The procedure for the classification process is outlined in Figure 2.
The tutorial will use the open source software Google Earth Engine (GEE) and its Python API. We will cover common tasks for data loading, image visualization and processing. The main steps of the classification process we developed for this tutorial with relevant software indicated in () are:

Image classification means mapping the values captured by remote sensors that are encoded as image digital levels to specific land cover types. <br>
Classifying remotely sensed data into a thematic map is a relevant task beacuse the resulting information is the basis for many environmental and socioeconomic applications. In this example, a crop map type is produced by applying a [*supervised*](https://en.wikipedia.org/wiki/Supervised_learning) [*Random Forest*](https://en.wikipedia.org/wiki/Random_forest) algorithm. 

<img src="./imgs/JensenClfProcess.PNG">

### Supervised Classification

A supervised classification method consists of training a classifier (algorithm) using (ground) truth data to classify  subsequently unseen data. <br>
The process flow in a supervised classification includes basically, the following steps:
* Collect the ground data. This data is used to train the classifier and validate its results.
* Train the classifier.
* Apply the classifier to produce a classified image.
* Assess the classification accuracy.

### Classification Scheme Definition

The classes and the detail of a classification scheme of a land cover map are determined by its intended use.  A classification scheme can have multiple levels of detail.  It is good practice to design a classification scheme with mutually exclusive and exhaustive classes of either land cover OR land use.  For different levels or scales the class details need to be determined in a way that they can be easily aggregated such as in a hierarchical structure.  

The classification scheme used in the runoff factor map delineation in the CAPRA flood model consists of a mixture of detailed land cover and land use classes as described in Table 1.  This is not an optimal scenario as overlap of classes can lead to non-exclusiveness of classes (e.g., impermeable surface and paved roads).  It is not consistent in the level of the hierarchical structure either.  The two level one forest land cover classes in the classification scheme are “natural forests” differentiated from “forests” which at the second level are further divided into  seeded or cultivated.  Further “cropped furrows”,” cereals”, “leguminous” and their subclasses can be grouped under agricultural or cropland at the first level. Roads, rangeland and rest (uncultivated) are pure land use classes, with class “rangeland” overlapping with class “pasture”.  

In general it is a lot more difficult to derive land use from a remotely sensed image than it is to detect land cover, as land cover describes the surface material and therefore has a direct linkage to the spectral reflectance behavior of the material (e.g. land cover grassland vs. land use classes pasture, football field, city park).  Land use classification of detailed agricultural classes as suggested by the CAPRA classification scheme requires a very good knowledge and expertise of a region’s agricultural practices.  

An important component of a land-cover classification procedure is the proper choice and delineation of training sites, used to train the computer in pattern recognition.  It is crucial to develop a database of reference points with reliable ground cover or use information at the spatial scale and thematic detailed at which the mapping procedure will be performed.  Preferably such reference points are acquired in the field (in situ) or from high resolution aerial photography.  For each of the different land cover types of interest a set of representative samples would be collected using a GPS (or digitizing technique).  

In our case we do not have a database of ground reference points or aerial photography, and we do not have an intimate knowledge of agricultural practices of the region, therefore we will refer to the U.S. Geological Survey Land Use/Land Cover Classification System for use with Remote Sensor Data (Tbl. 2).  

<img src="./imgs/LCclassTable.PNG">

For this tutorial we will use classes based on level one of the USGS classification system.  The classes of interest for our mapping application are: Water, Forest Land, Rangeland, Agricultural Land, Barren Land, and Urban or Built-up Land.  
Since the classification scheme is a mix of land cover and land use we will rename classes with land use character to their corresponding land cover.  We also want to avoid spaces in the class names since we are going to use the names in code and spaces can be problematic.  The class names we are going to use instead are:

(1)	water 
(2)	forestLand
(3)	cropLand
(4)	buildUpLand

#### Question:
#### (1) Which of the level one classes of the U.S. Geological Survey Land Use/Land Cover Classification System for use with remote                  sensor Data do you expect to be present in your ROI (google it).

### Remote Sensing Process

One of the more common procedures to generate thematic land cover maps of a region is by applying pattern-recognition algorithms to satellite imagery in a digital image processing environment.  The use of remote sensing classification methods involve three general steps, (1) the selection and pre-processing of imagery adequate for the mapping of land cover classes of interest at their appropriate scales, (2) extraction and evaluation of spectral signatures and their statistical separability for all land cover classes of interest and development of a classification method and (3) evaluation and documentation of the mapping process and the final map product.   

(1)	Discrimination, and mapping of various land use/land cover classes depends on the spatial resolution (pixel size); the spectral resolution (number of bands or channels and their bandwidth); the radiometric resolution (range of discrete brightness values); and the temporal resolution (return time of the sensor) in relation to the classification scheme and minimum mapping unit or appropriate scale (Teillet et al., 1997 ,Rao et al., 2007).  Since the limitations of a mapping project are often constrained by its budget, the most economic approach to consider is to use data free of charge processed with open source image processing software, which is the a solution demonstrated in this tutorial.  

A large Landsat archive of 45 years of image acquisitions is available and open to the public. Landsat sensor specific information are accessible through the data portal of USGS (http://landsat.usgs.gov/Landsat_Search_and_Download.php).  For this exercise we are going to use Landsat 8 images. The Scan Line Corrector (SLC) in the Landsat 7 failed in in April of 2003 resulting in imagery with significant geometric error and considerable areas of no data value in each scene.  The Landsat 8 data source is adequate for the classes we identified in the classification scheme. Google Earth Engine (GEE) contains a variety of Landsat specific processing methods. Specifically, there are methods to compute at-sensor radiance, top-of-atmosphere reflectance, surface reflectance, cloud score and cloud-free composites.   
    
(2)	The extraction and evaluation of spectral signatures of all land cover classes of interest requires a training dataset.  In this exercise a set of training points for each class will be digitized on screen using the Google Earth application.  For the digitized sampling points spectral characteristics will then be extracted from the pre-processed imagery.  The variables we are interested in evaluating are reflectance values.  In addition we will consider derivative information, the Normalized Difference Vegetation Index (NDVI) and tasseled cap or Kauth-Thomas transformation which is based on the relationship of spectral characteristics to soil brightness, moisture content and vegetation cover.  The pre-processing and preparation of all data and the extraction of variable values for each sample of the training data set will be performed in Google Earth and GEE.

The classification process is guided by the evaluation of the separability of classes for different combinations of reflectance bands and other derived variables based on the training set.  Separability analysis and feature selection are frequently based on descriptive class parameters such as the class specific mean vector and variance-covariance matrices.  For this tutorial instead of using parametric measures to evaluate class-separability, we will evaluate the effectiveness of a recursive partitioning algorithm, a non-parametric classification procedure (see pg. 38 in landCoverClassification_Theory.pdf).  Performance of different classifiers is assessed for all evaluated models based on accuracy measures derived from confusion matrices of the classified training dataset.  This initial evaluation of accuracies has a bias towards overestimating accuracies, and will only be used to select the model with the best fit.  The analysis will provide two important components for the final classification, a list of variables that are most suitable to differentiate the classes of interest, and the decision rules or classification tree (the model or classifier), that will be applied to all pixels of the entire area of interest, the Dominican Republic.  This part of the analysis will be performed in GEE Python API.      

(3)	The final step of the mapping project is the accuracy assessment of the final classification results (the final map).  For this purpose we will use a stratified random sampling design (see pg. 45 in landCoverClassification_Theory.pdf) applied to the sampling frame of all classified image elements (pixels).  For lack of ground reference data we will use Google Earth (or Google maps) as reference source to determine classification accuracy.


In [1]:
import ee
from IPython.display import Image
ee.Initialize()
# Load the image
# load the image collection and filter
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

CBD = ee.FeatureCollection("USDOS/LSIB/2013")
boundary = CBD.filterMetadata('name', 'equals', 'DOMINICAN REPUBLIC')

image = ee.Image(l8sr
            .filterDate('2018-01-01', '2019-04-30')
            .filterBounds(boundary)
            .sort('CLOUD_COVER')   
            .first()
)

For creating simple cloud-free Landsat composites, Earth Engine provides the ee.Algorithms.Landsat.simpleComposite() method. This method selects a subset of scenes at each location, converts to TOA reflectance, applies the simple cloud score and takes the median of the least cloudy pixels. This example creates a simple composite using default parameters and compares it to a composite using custom parameters for the cloud score threshold and the percentile:

In [6]:
import pprint
# load the image collection and filter
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2018-01-01', '2018-12-31').filterBounds(boundary)                      

composite = ee.Algorithms.Landsat.simpleComposite(l8sr)
#pprint(composite.getInfo())

In [2]:
import pprint
# Load tables into feature collection
#points = ee.FeatureCollection('ft:10q302Kafv_V_Le-jm1Cj274Ejz0kTsSWTKNxf3br').remap([1, 2, 3,4], [0, 1, 2, 3], "class")
## https://fusiontables.google.com/data?docid=10q302Kafv_V_Le-jm1Cj274Ejz0kTsSWTKNxf3br
# points = ee.FeatureCollection('ft:1p-YuR8JdqopYb-LrUxb43xzGHQyKmhGr1EJU3wku').remap([1, 2, 3,4], [0, 1, 2, 3], "name")
## find table in https://fusiontables.google.com/data?docid=1Axs89HgE3yIgPoTMxTjL965OYgtBrp2i1O2waz29#rows:id=1
points = ee.FeatureCollection('ft:1Axs89HgE3yIgPoTMxTjL965OYgtBrp2i1O2waz29')

#pprint.pprint(points.getInfo())

In [3]:
# Select the bands for training
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

In [4]:
# Sample the input imagery to get a FeatureCollection of training data.
training = image.select(bands).sampleRegions(points,['landcover'],30)
#pprint.pprint(training.getInfo())

In [5]:
# Make a Random Forest classifier and train it.
#rf = ee.Classifier.randomForest(10).train(training,["landcover"],bands)
cart = ee.Classifier.cart().train(training,["landcover"],bands)
#print(rf)

In [6]:
# Classify the input imagery.
#result = image.select(bands).classify(rf)
result_cart = image.select(bands).classify(cart)
#print(result)

In [7]:
region = boundary.geometry().bounds().getInfo()['coordinates'][0]
task = ee.batch.Export.image.toDrive(
  image = result_cart,
  description = "LCimage",
  #assetId = 'users/ximenamesa/LC' + "DR",
  maxPixels = 100000000,   
  #region = region,
  scale = 2)
task.start()
print("Task started")

EEException: Classifier.train, argument 'classProperty': Invalid type. Expected: String. Actual: List<String>.

In [9]:
#LCclip = result.clip(boundary)
from IPython.display import Image
Image(url=result_cart.getThumbUrl({'min': 1, 'max': 4}))

EEException: Classifier.train, argument 'classProperty': Invalid type. Expected: String. Actual: List<String>.

In [ ]:
clas_col = ','.join(['red','green','blue','pink'])   
Image(url = result.getThumbUrl({'min': 1, 'max': 4, 'palette':clas_col}))

Classification result could be improved by adding more data such as vegetation indices and textural features. You may try to perform a new classification considering those features.